# Create Initial drought features

Estamos procesando la informacion de las sequias para hacerlo un modelo supervisado. En la explicación del [MSM](https://smn.conagua.gob.mx/es/climatologia/monitor-de-sequia/monitor-de-sequia-en-mexico), explican que desde el 2016, la definición de lo que consideran sequía cambia, por esta razón tomaremos los datos a partir de esa fecha.

## Imports

In [1]:
import pandas as pd
import warnings

from dateutil.relativedelta import relativedelta

from src.data.utils import (
    get_general_path, join_paths, zeroes_to_cve, save_as_pickle, save_dataframe
)

## Constants

In [2]:
INTERIM_DATA_PATH = 'data/interim'
PROCESSED_DROUGTH_DATA_FILE = 'processed_drought_data.parquet'

FEATURES_FILE = 'initial_features.parquet'

DATE_NAME = 'date'
NEW_DATE = 'standard_date'
THRESHOLD_DATE = '2016-01-01'

## Config

In [3]:
warnings.filterwarnings('ignore')

## Functions

In [4]:
def calculate_rolling_window_features(data, days, date_col='DATE', column_value='DROUGHT_INDEX'):
    df = data.copy()
    rolling_df = df.rolling(f'{days}D', on=date_col, min_periods=days//30-2, closed='left')
    rolling_df_target = rolling_df[column_value]

    df[f'{column_value}__mean__last{days}_days'] = rolling_df_target.mean()
    df[f'{column_value}__std__last{days}_days'] = rolling_df_target.std()
    df[f'{column_value}__max__last{days}_days'] = rolling_df_target.max()
    df[f'{column_value}__min__last{days}_days'] = rolling_df_target.min()
    df[f'{column_value}__median__last{days}_days'] = rolling_df_target.median()
    df[f'{column_value}__kurtosis__last{days}_days'] = rolling_df_target.kurt()
    df[f'{column_value}__skewness__last{days}_days'] = rolling_df_target.skew()
    df[f'{column_value}__range__last{days}_days'] = (
            df[f'{column_value}__max__last{days}_days'] -
            df[f'{column_value}__min__last{days}_days']
    )
    df[f'{column_value}__mean_vs_median__last{days}_days'] = (
            df[f'{column_value}__mean__last{days}_days'] -
            df[f'{column_value}__median__last{days}_days']
    )
    features_cols = [col for col in df.columns if '__' in col]
    return df[features_cols]

def calculate_stable_drought(df):
    stable_df = df.copy()
    stable_df['is_stable'] = (stable_df.num_drought_index.diff() == 0).astype('float')
    stable_df['val_groups'] = (stable_df.is_stable != stable_df.is_stable.shift()).cumsum()
    stable_df['date_differences'] = stable_df.standard_date.diff().dt.days
    stable_df['months_stable'] = stable_df.groupby('val_groups').date_differences.cumsum()/30
    return stable_df[['is_stable', 'months_stable']]

## Read data

In [5]:
general_path = get_general_path()
processed_drought_data_path = join_paths(general_path, INTERIM_DATA_PATH, PROCESSED_DROUGTH_DATA_FILE)
processed_drought_data = pd.read_parquet(processed_drought_data_path)

## Process Data

In [6]:
useless_columns = [
    'CVE_CONCATENADA',
    'CVE_ENT',
    'CVE_MUN',
    'NOMBRE_MUN',
    'ENTIDAD',
    'ORG_CUENCA*',
    'CON_CUENCA',
]
data = processed_drought_data.drop(useless_columns, axis=1)

In [7]:
data_num_drought_index__clv_oc = data.groupby(['CLV_OC','standard_date']).agg(
    num_drought_index__clv_oc__mean=('num_drought_index', 'mean'),
    num_drought_index__clv_oc__std=('num_drought_index', 'std'),
    num_drought_index__clv_oc__max=('num_drought_index', 'max'),
    num_drought_index__clv_oc__min=('num_drought_index', 'min'),
    num_drought_index__clv_oc__median=('num_drought_index', 'median'),
)

data_num_drought_index__cve_conc = data.groupby(['CVE_CONC','standard_date']).agg(
    num_drought_index__cve_conc__mean=('num_drought_index', 'mean'),
    num_drought_index__cve_conc__std=('num_drought_index', 'std'),
    num_drought_index__cve_conc__max=('num_drought_index', 'max'),
    num_drought_index__cve_conc__min=('num_drought_index', 'min'),
    num_drought_index__cve_conc__median=('num_drought_index', 'median'),
)

In [8]:
data_with_features = data.merge(
    data_num_drought_index__clv_oc.reset_index(),
    on=['CLV_OC', 'standard_date'],
    how='left',
).merge(
    data_num_drought_index__cve_conc.reset_index(),
    on=['CVE_CONC', 'standard_date'],
    how='left'
).set_index(data.index)

In [9]:
data_drougt_index_3m = data.groupby('mun_id').apply(
    calculate_rolling_window_features,
    days=100,
    date_col='standard_date',
    column_value='num_drought_index'
).reset_index().set_index('mun_id__date').drop('mun_id', axis=1)

data_drougt_index_6m = data.groupby('mun_id').apply(
    calculate_rolling_window_features,
    days=190,
    date_col='standard_date',
    column_value='num_drought_index'
).reset_index().set_index('mun_id__date').drop('mun_id', axis=1)

data_stability_drought_index = data.groupby('mun_id').apply(
    calculate_stable_drought
).reset_index().set_index('mun_id__date').drop('mun_id', axis=1)

In [10]:
features = pd.concat([data_with_features, data_drougt_index_3m, data_drougt_index_6m, data_stability_drought_index], axis=1)

In [11]:
# Since not all values are valid to compute since there is a problem with the information, we need this filter:
final_features = features[features['date'] >  pd.to_datetime(THRESHOLD_DATE)]
final_features.drop(['CLV_OC', 'CVE_CONC', 'date'], axis=1, inplace=True)

## Results

In [12]:
final_features

,mun_id,num_drought_index,standard_date,num_drought_index__clv_oc__mean,num_drought_index__clv_oc__std,num_drought_index__clv_oc__max,num_drought_index__clv_oc__min,num_drought_index__clv_oc__median,num_drought_index__cve_conc__mean,num_drought_index__cve_conc__std,...,num_drought_index__std__last190_days,num_drought_index__max__last190_days,num_drought_index__min__last190_days,num_drought_index__median__last190_days,num_drought_index__kurtosis__last190_days,num_drought_index__skewness__last190_days,num_drought_index__range__last190_days,num_drought_index__mean_vs_median__last190_days,is_stable,months_stable
mun_id__date,,,,,,,,,,,,,,,,,,,,,
01_001__20160115,01_001,0.0,2016-01-15,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,-3.000000,0.000000,0.0,0.000000,1.0,44.233333
01_002__20160115,01_002,0.0,2016-01-15,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,-3.000000,0.000000,0.0,0.000000,1.0,44.233333
01_003__20160115,01_003,0.0,2016-01-15,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,-3.000000,0.000000,0.0,0.000000,1.0,44.233333
01_004__20160115,01_004,0.0,2016-01-15,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,-3.000000,0.000000,0.0,0.000000,1.0,44.233333
01_005__20160115,01_005,0.0,2016-01-15,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,-3.000000,0.000000,0.0,0.000000,1.0,44.233333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32_054__20240615,32_054,1.0,2024-06-15,2.380952,1.096597,4.0,0.0,2.0,2.191489,1.244786,...,0.577350,2.0,0.0,1.0,0.654545,-0.062984,2.0,-0.166667,0.0,1.033333
32_055__20240615,32_055,3.0,2024-06-15,2.429448,0.859118,4.0,0.0,3.0,2.064516,0.749006,...,0.000000,3.0,3.0,3.0,-3.000000,0.000000,0.0,0.000000,1.0,11.766667
32_056__20240615,32_056,2.0,2024-06-15,2.429448,0.859118,4.0,0.0,3.0,2.064516,0.749006,...,0.937437,3.0,1.0,1.5,-1.930559,0.382554,2.0,0.333333,1.0,0.600000


## Conclusion

In [13]:
feature_files = join_paths(general_path, INTERIM_DATA_PATH, FEATURES_FILE)
save_dataframe(feature_files, final_features, 'parquet')

data was saved into `/mnt/c/Users/dhdzm/Documents/projects/seguia/seguia/src/data/../../data/interim/initial_features.parquet`.
